In [31]:
import pandas as pd
import numpy as np
import json
import spacy
from scipy.spatial.distance import cdist

In [10]:
import spacy

In [11]:
!python -m spacy download en_core_web_md

/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named spacy


In [13]:
spacy.cli.download("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [14]:
nlp = spacy.load('en_core_web_md')

In [26]:
data = pd.read_csv('/Users/nursevencan/Desktop/customer_data.csv')

In [27]:
# Define the attributes for the vector embeddings
attributes = ['Name', 'ID', 'Fellowships', 'City', 'Country', 'Bio','Company','Role','Success','Interest']

In [35]:
# Calculate the vector embeddings for each customer
import csv
embeddings = {}
with open('/Users/nursevencan/Desktop/customer_data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        text = ''
        for attr in attributes:
            value = row[attr]
            if isinstance(value, float):
                value = str(value)
            text += value + ' '
        doc = nlp(text)
        embeddings[row['ID']] = doc.vector.tolist()

In [36]:
# Calculate the cosine similarity matrix between all pairs of customers
keys = list(embeddings.keys())
distances = cdist(list(embeddings.values()), list(embeddings.values()), metric='cosine')
similarity_matrix = pd.DataFrame(distances, columns=keys, index=keys)

In [37]:
# Select the k-nearest neighbors for each customer and recommend the top matches
k = 10
matches = {}
for key in embeddings.keys():
    nearest_neighbors = similarity_matrix.nlargest(k, key)[key]
    matches[key] = list(nearest_neighbors.index)

In [38]:
# Save the matches to a JSON file
with open('matches.json', 'w') as f:
    json.dump(matches, f)